## Modify the softmax structure - weight scaling
Try to differentiate the similar weights, TF-IDF-like approach.
If the word is similarly occuring in all classes, it has to have the weight lowered.

Otherwise, if the probability for one class is significantly higher than for the other ones,
the weight is increased by a significant margin

In [1]:
# Load the structure
import pickle
import pandas as pd
import dask.dataframe as dd
from tqdm import tqdm
import numpy as np
import copy
import os
import toml
import re
import itertools
from text_cleaner import *
import operator
from collections import Counter

def extract_hashtags(post_text):
    HASH_RE = re.compile(r"\#\w+")
    out_list = re.findall(HASH_RE, post_text)
    return out_list

with open("softmax_full_word_trait_array.pickle", "rb") as f:
    word_df = pickle.load(f)

In [2]:
# Show the loaded DataFrame
word_df

zainspirowany  wczorajszym      wywiadem  odnośnie   relacji  \
innocent    0       0.000021     0.000043  2.125173e-05  0.000043  0.000149   
            1       0.000000     0.000013  0.000000e+00  0.000013  0.000243   
            2       0.000002     0.000019  0.000000e+00  0.000045  0.000112   
            3       0.000006     0.000034  9.158188e-07  0.000037  0.000173   
            4       0.000000     0.000016  0.000000e+00  0.000055  0.000079   
...                      ...          ...           ...       ...       ...   
allocentric 0       0.000016     0.000033  3.259984e-05  0.000049  0.000228   
            1       0.000009     0.000033  0.000000e+00  0.000033  0.000164   
            2       0.000000     0.000021  0.000000e+00  0.000036  0.000088   
            3       0.000005     0.000027  0.000000e+00  0.000046  0.000145   
            4       0.000003     0.000032  0.000000e+00  0.000026  0.000194   

               chciałem  przekazać     okres  kwarantanny   świetny  ...  \
innocent    0  0.000106   0.000064  0.000149     0.000149  0.000128  ...   
            1  0.000013   0.000013  0.000040     0.000040  0.000013  ...   
            2  0.000074   0.000045  0.000100     0.000045  0.000095  ...   
            3  0.000026   0.000030  0.000132     0.000039  0.000071  ...   
            4  0.000008   0.000040  0.000079     0.000047  0.000040  ...   
...                 ...        ...       ...          ...       ...  ...   
allocentric 0  0.000081   0.000081  0.000081     0.000065  0.000081  ...   
            1  0.000021   0.000021  0.000270     0.000048  0.000061  ...   
            2  0.000047   0.000031  0.000109     0.000026  0.000109  ...   
            3  0.000046   0.000028  0.000085     0.000042  0.000076  ...   
            4  0.000016   0.000062  0.000055     0.000055  0.000058  ...   

               #piekewdomu     #dish  #kremzcukinii  #zucchina  \
innocent    0     0.000000  0.000000       0.000000   0.000000   
            1     0.000000  0.000000       0.000000   0.000000   
            2     0.000000  0.000000       0.000000   0.000000   
            3     0.000000  0.000000       0.000000   0.000000   
            4     0.000040  0.000016       0.000008   0.000016   
...                    ...       ...            ...        ...   
allocentric 0     0.000000  0.000000       0.000000   0.000000   
            1     0.000000  0.000000       0.000000   0.000000   
            2     0.000000  0.000000       0.000000   0.000000   
            3     0.000006  0.000002       0.000001   0.000002   
            4     0.000000  0.000000       0.000000   0.000000   

               #pranzoitaliano  #tradycyjnejedzenie    #tatar  #meatlover  \
innocent    0         0.000000             0.000000  0.000000    0.000000   
            1         0.000000             0.000000  0.000000    0.000000   
            2         0.000000             0.000000  0.000000    0.000000   
            3         0.000000             0.000000  0.000000    0.000000   
            4         0.000008             0.000008  0.000008    0.000008   
...                        ...                  ...       ...         ...   
allocentric 0         0.000000             0.000000  0.000000    0.000000   
            1         0.000000             0.000000  0.000000    0.000000   
            2         0.000000             0.000000  0.000000    0.000000   
            3         0.000001             0.000001  0.000001    0.000001   
            4         0.000000             0.000000  0.000000    0.000000   

               #zachcianki  #pierogizkapusta  
innocent    0     0.000000          0.000000  
            1     0.000000          0.000000  
            2     0.000000          0.000000  
            3     0.000000          0.000000  
            4     0.000008          0.000008  
...                    ...               ...  
allocentric 0     0.000000          0.000000  
            1     0.000000          0.000000  
            

In [3]:
# Get all traits/archetypes
trait_list = word_df.index.values
trait_list = list(dict.fromkeys([x[0] for x in trait_list]))
print(trait_list)

['innocent', 'sage', 'explorer', 'outlaw', 'magician', 'hero', 'lover', 'jester', 'everyman', 'caregiver', 'ruler', 'creator', 'dominant', 'submissive', 'maximalist', 'minimalist', 'inspiring', 'systematic', 'discovering', 'conservative', 'verifying', 'overlooking', 'sharpening', 'harmonic', 'empathic', 'matter_of_fact', 'brave', 'protective', 'generous', 'thrifty', 'favourable', 'balanced', 'sensuality', 'intelligent', 'believe', 'egocentric', 'allocentric']


In [5]:
# Normalization method
def min_max_normalize(df_to_normalize):
    normalized_df = df_to_normalize
    cols = normalized_df.columns.tolist()
    for col in tqdm(cols):
        normalized_df[col] = (normalized_df[col] - normalized_df[col].min()) / (normalized_df[col].max() - normalized_df[col].min())
    return normalized_df

norm_list = []
for trait in trait_list:
    print(f"Trait: {trait}")
    normalized_trait = min_max_normalize(copy.deepcopy(word_df.loc[trait]))
    tmp_trait = normalized_trait
    norm_list.append(tmp_trait)
    
norm_word_df = pd.concat(norm_list, keys=trait_list)

  0%|          | 347/225423 [00:00<02:04, 1800.70it/s]

Trait: innocent


  0%|          | 218/225423 [00:00<01:43, 2172.76it/s]

Trait: sage


  0%|          | 412/225423 [00:00<01:48, 2068.53it/s]

...

  0%|          | 200/225423 [00:00<01:52, 1995.51it/s]

Trait: egocentric


  0%|          | 231/225423 [00:00<01:37, 2305.18it/s]

Trait: allocentric


100%|██████████| 225423/225423 [01:45<00:00, 2130.09it/s]


In [6]:
norm_word_df

zainspirowany  wczorajszym  wywiadem  odnośnie   relacji  \
innocent    0       1.000000     1.000000  1.000000  0.693486  0.426116   
            1       0.000000     0.000000  0.000000  0.000000  1.000000   
            2       0.111937     0.191227  0.000000  0.757875  0.200239   
            3       0.301657     0.703053  0.043094  0.553207  0.574807   
            4       0.000000     0.080213  0.000000  1.000000  0.000000   
...                      ...          ...       ...       ...       ...   
allocentric 0       1.000000     0.942079  1.000000  1.000000  1.000000   
            1       0.557682     1.000000  0.000000  0.322751  0.539284   
            2       0.000000     0.000000  0.000000  0.449722  0.000000   
            3       0.283110     0.461321  0.000000  0.880241  0.409001   
            4       0.198704     0.925341  0.000000  0.000000  0.758377   

               chciałem  przekazać     okres  kwarantanny   świetny  ...  \
innocent    0  1.000000   1.000000  1.000000     1.000000  1.000000  ...   
            1  0.056698   0.000000  0.000000     0.009714  0.000000  ...   
            2  0.669417   0.630885  0.549016     0.053189  0.716246  ...   
            3  0.180353   0.332996  0.844126     0.000000  0.508228  ...   
            4  0.000000   0.517986  0.356370     0.073563  0.228374  ...   
...                 ...        ...       ...          ...       ...  ...   
allocentric 0  1.000000   1.000000  0.123193     1.000000  0.459871  ...   
            1  0.076809   0.000000  1.000000     0.574663  0.045621  ...   
            2  0.465693   0.163556  0.250160     0.000000  1.000000  ...   
            3  0.458651   0.107442  0.141233     0.397878  0.353671  ...   
            4  0.000000   0.668910  0.000000     0.742059  0.000000  ...   

               #piekewdomu  #dish  #kremzcukinii  #zucchina  #pranzoitaliano  \
innocent    0          0.0    0.0            0.0        0.0              0.0   
            1          0.0    0.0            0.0        0.0              0.0   
            2          0.0    0.0            0.0        0.0              0.0   
            3          0.0    0.0            0.0        0.0              0.0   
            4          1.0    1.0            1.0        1.0              1.0   
...                    ...    ...            ...        ...              ...   
allocentric 0          0.0    0.0            0.0        0.0              0.0   
            1          0.0    0.0            0.0        0.0              0.0   
            2          0.0    0.0            0.0        0.0              0.0   
            3          1.0    1.0            1.0        1.0              1.0   
            4          0.0    0.0            0.0        0.0              0.0   

               #tradycyjnejedzenie  #tatar  #meatlover  #zachcianki  \
innocent    0                  0.0     0.0         0.0          0.0   
            1                  0.0     0.0         0.0          0.0   
            2                  0.0     0.0         0.0          0.0   
            3                  0.0     0.0         0.0          0.0   
            4                  1.0     1.0         1.0          1.0   
...                            ...     ...         ...          ...   
allocentric 0                  0.0     0.0         0.0          0.0   
            1                  0.0     0.0         0.0          0.0   
            2                  0.0     0.0         0.0          0.0   
            3                  1.0     1.0         1.0          1.0   
            4                  0.0     0.0         0.0          0.0   

               #pierogizkapusta  
innocent    0               0.0  
            1               0.0  
            2               0.0  
            3               0.0  
            4               1.0  
...                         ...  
allocentric 0               0.0  
            1               0.0  
            2               0.0  
            3               1.0  
            4

In [7]:
from scipy.special import softmax

def get_trait_dot_product(post_text: str, word_map: list, word_dataframe: pd.DataFrame) -> list:
    # Filter out the text
    filtered_post = remove_stopwords(clean_up_text(post_text))
    filtered_post += extract_hashtags(post_text)
    
    # Create a vector for dot product vector
    post_vector = [0] * len(word_map)
    
    # Calculate word occurrences
    word_ctr = Counter(filtered_post)
    
    for word, freq in word_ctr.items():
        if word in word_map:
            post_vector[word_map.index(word)] = freq
    
    # Calculate dot product for a given text
    word_dot = word_dataframe.dot(post_vector)
    
    out_vec = pd.Series()
    for trait in trait_list:
        out_vec = out_vec.append(pd.Series([np.argmax(softmax(word_dot.loc[trait]))], index=[trait]))
    
    return out_vec

# Trait accuracy - round the results
def natural_round(x: float) -> int:
    out = int(x // 1)
    return out + 1 if (x - out) >= 0.5 else out

def accuracy_per_trait(input_vector: pd.Series, annotated_vector: pd.Series) -> np.array:
    out_array = np.array([0] * 37, dtype=np.int)
    for i in range(len(out_array)):
        if input_vector[i] == annotated_vector[i]:
            out_array[i] = 1
    return out_array

In [8]:
# Load the .csv with archetypes
arch_df = pd.read_csv('archetypes_pl_new.csv', index_col=0)

# Save the order of columns
trait_list = arch_df.columns.tolist()

# Show the table header and column list
print(trait_list)
arch_df.head()

['innocent', 'sage', 'explorer', 'outlaw', 'magician', 'hero', 'lover', 'jester', 'everyman', 'caregiver', 'ruler', 'creator', 'dominant', 'submissive', 'maximalist', 'minimalist', 'inspiring', 'systematic', 'discovering', 'conservative', 'verifying', 'overlooking', 'sharpening', 'harmonic', 'empathic', 'matter_of_fact', 'brave', 'protective', 'generous', 'thrifty', 'favourable', 'balanced', 'sensuality', 'intelligent', 'believe', 'egocentric', 'allocentric']


,innocent,sage,explorer,outlaw,magician,hero,lover,jester,everyman,caregiver,...,protective,generous,thrifty,favourable,balanced,sensuality,intelligent,believe,egocentric,allocentric
id,,,,,,,,,,,,,,,,,,,,,
marek_grodzki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,2.0,3.0,4.0,4.0,3.0,4.0,4.0,3.0,0.0,0.0
vege_style_life,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,0.0,3.0
oliwka__2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,2.0,2.0,0.0,3.0,1.0,2.0,4.0,1.0,0.0,3.0
z_przestrzeni_serca,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0.0,4.0,4.0,3.0,4.0,4.0,0.0,1.0
zaradne_warsztaty,3.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,3.0,4.0,...,3.0,4.0,0.0,2.0,2.0,4.0,2.0,3.0,1.0,3.0


In [9]:
# Table preprocessing - replace all NaN with 2 (Unrelated/Don't know class), replace 0-5 values with the ones in range -1.0 - 1.0
arch_df = arch_df.fillna(2)

# Remove duplicated annotations, to exclude conflicting entries
arch_df = arch_df[~arch_df.index.duplicated(keep='first')]

# Print the head of the dataset after modification
arch_df.head()

,innocent,sage,explorer,outlaw,magician,hero,lover,jester,everyman,caregiver,...,protective,generous,thrifty,favourable,balanced,sensuality,intelligent,believe,egocentric,allocentric
id,,,,,,,,,,,,,,,,,,,,,
marek_grodzki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,2.0,3.0,4.0,4.0,3.0,4.0,4.0,3.0,0.0,0.0
vege_style_life,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,0.0,3.0
oliwka__2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,2.0,2.0,0.0,3.0,1.0,2.0,4.0,1.0,0.0,3.0
z_przestrzeni_serca,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0.0,4.0,4.0,3.0,4.0,4.0,0.0,1.0
zaradne_warsztaty,3.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,3.0,4.0,...,3.0,4.0,0.0,2.0,2.0,4.0,2.0,3.0,1.0,3.0


In [10]:
# Check if a user has a non-empty directory in the dataset, otherwise delete the user from the list
available_arch_df = copy.deepcopy(arch_df)
posts = []

BASE_DIR = "instagram_cleared"

# Iterate over whole DataFrame
for i, row in tqdm(arch_df.iterrows()):
    profile_posts = []
    profile_hashtags = []
    
    # Get all posts per profile
    profile_path = os.path.join(BASE_DIR, i)
    for file in os.listdir(profile_path):
        if not file.endswith(".toml"):
            with open(os.path.join(profile_path, file), "r") as post_f:
                read_text = post_f.read()
                profile_posts.append(remove_stopwords(clean_up_text(read_text)))
                profile_hashtags.append(extract_hashtags(read_text))

    # Merge lists - a single list for a single influencer
    profile_hashtags = list(itertools.chain.from_iterable(profile_hashtags))
    posts.append(list(itertools.chain.from_iterable([profile_posts, [profile_hashtags]])))

685it [00:20, 34.13it/s]


In [11]:
# Map usernames to indices
users = list(available_arch_df.index.values)
user_indices = {k: users.index(k) for k in users}

In [12]:
# Create word map
norm_word_map = norm_word_df.columns.tolist()

In [13]:
pbar = tqdm(arch_df.iterrows())
accuracy = 0

# Out accuracy vector
total_accuracy = np.array([0] * 37, dtype=np.int)

for idx, row in pbar:
    user_text = list(itertools.chain.from_iterable(posts[users.index(idx)]))
    user_text = " ".join(user_text)
    sim_output = get_trait_dot_product(user_text, norm_word_map, norm_word_df)
    user_accuracy = accuracy_per_trait(sim_output, row)
    total_accuracy += user_accuracy
    pbar.set_description(f"Average accuracy: {round(np.mean(np.divide(total_accuracy, users.index(idx)+1))*100, 2)}")

0it [00:00, ?it/s]<ipython-input-7-4ffef00e153a>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  out_vec = pd.Series()
Average accuracy: 97.8: : 685it [20:23,  1.79s/it] 


In [14]:
# Show total accuracy
scaled_accuracy = np.divide(total_accuracy, len(arch_df))
avg_accuracy = np.mean(scaled_accuracy)

print("--- ACCURACY ON WHOLE DATASET ---")

print(f"Average dataset accuracy: {round(avg_accuracy*100, 2)}%")
print("Accuracy per trait:")
for i in range(len(trait_list)):
    print(f"{trait_list[i]}: {round(scaled_accuracy[i] * 100, 2)}%")

--- ACCURACY ON WHOLE DATASET ---
Average dataset accuracy: 97.8%
Accuracy per trait:
innocent: 98.54%
sage: 97.52%
explorer: 97.66%
outlaw: 99.56%
magician: 99.56%
hero: 98.98%
lover: 98.69%
jester: 97.96%
everyman: 96.06%
caregiver: 98.39%
ruler: 96.64%
creator: 97.96%
dominant: 94.6%
submissive: 98.25%
maximalist: 97.37%
minimalist: 97.52%
inspiring: 99.27%
systematic: 95.47%
discovering: 97.37%
conservative: 99.85%
verifying: 95.18%
overlooking: 96.35%
sharpening: 98.54%
harmonic: 98.83%
empathic: 99.27%
matter_of_fact: 94.01%
brave: 98.25%
protective: 98.1%
generous: 99.27%
thrifty: 99.42%
favourable: 98.25%
balanced: 98.98%
sensuality: 98.1%
intelligent: 95.18%
believe: 97.08%
egocentric: 98.54%
allocentric: 98.1%


In [16]:
# Save AGDS to file
norm_word_df.to_pickle("softmax_norm_full_word_trait_array.pickle")